Abstractive Text Summarization with PEGASUS (Pre-training with Extracted Gap-sentences for Abstractive SUmmarization)

Fine-tuning PEGASUS with WikiHow Knowledge base

In [ ]:
# Cloning the requirements, setup, readme, and pegasus pre-trained model files

In [ ]:
!pwd
!git clone https://github.com/ManuMahadevaswamy/PEGASUS

/content
Cloning into 'PEGASUS'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 158 (delta 48), reused 148 (delta 45), pack-reused 0
Receiving objects: 100% (158/158), 348.10 KiB | 4.35 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
cd PEGASUS/

/content/PEGASUS


In [ ]:
!export PYTHONPATH=.

In [ ]:
# Installing all the necessary libraries as per the requirements.txt

In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 10.3MB/s 
     |████████████████████████████████| 8.6MB 11.6MB/s 
     |████████████████████████████████| 1.4MB 23.4MB/s 
     |████████████████████████████████| 3.5MB 39.0MB/s 
     |████████████████████████████████| 411.0MB 44kB/s 
     |████████████████████████████████| 110.5MB 36kB/s 
     |████████████████████████████████| 358kB 33.2MB/s 
     |████████████████████████████████| 184kB 45.9MB/s 
     |████████████████████████████████| 5.3MB 41.2MB/s 
     |████████████████████████████████| 368kB 37.9MB/s 
     |████████████████████████████████| 307kB 47.1MB/s 
     |████████████████████████████████| 655kB 42.5MB/s 
     |████████████████████████████████| 81kB 7.8MB/s 
     |████████████████████████████████| 983kB 41.4MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 512kB 44.4MB/s 
     |████████████████████████████████| 3.8MB 36.2MB/s

In [ ]:
# creating a folder to save checkpoints

In [ ]:
mkdir ckpt

In [ ]:
cd /content/PEGASUS/ckpt

/content/PEGASUS/ckpt


In [ ]:
mkdir pegasus_ckpt

In [ ]:
cd /content/PEGASUS/

/content/PEGASUS


In [ ]:
# Copying the Pre-trained, Vocabulary models and checkpoints from Google Cloud Storage using gsutil commands

In [ ]:
!gsutil -m cp -r gs://pegasus_ckpt/*.model ckpt/pegasus_ckpt
!gsutil -m cp -r gs://pegasus_ckpt/*.vocab ckpt/pegasus_ckpt
!gsutil -m cp -r gs://pegasus_ckpt/*.meta ckpt/pegasus_ckpt
!gsutil -m cp -r gs://pegasus_ckpt/*.index ckpt/pegasus_ckpt
!gsutil -m cp -r gs://pegasus_ckpt/checkpoint ckpt/pegasus_ckpt
!gsutil -m cp -r gs://pegasus_ckpt/*.data-00000-of-00001 ckpt/pegasus_ckpt

Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
/ [1/1 files][  1.8 MiB/  1.8 MiB] 100% Done                                    
Operation completed over 1 objects/1.8 MiB.                                      
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
/ [1/1 files][  1.7 MiB/  1.7 MiB] 100% Done                                    
Operation completed over 1 objects/1.7 MiB.                                      
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
\ [1/1 files][220.4 MiB/220.4 MiB] 100% Done                                    
Operation completed over 1 objects/220.4 MiB.                                    
Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
/ [1/1 files][ 16.7 KiB/ 16.7 KiB] 100% Done                                    
Operation completed over 1 objects/16.7 KiB.                                     
Copying gs://pegasus_ckpt/checkpoint...
/ [1/1 files][   93.0 B/   93.0 B] 100% Done                         

In [ ]:
cd /content/

/content


In [ ]:
!pip3 install -e PEGASUS/

Obtaining file:///content/PEGASUS
  Running setup.py develop for pegasus


In [ ]:
cd /content/PEGASUS/

/content/PEGASUS


Importing fine-tuning downstream data(WikiHow Knowlwdge Base into Colab environment)

In [ ]:
# First upload the csv files before importing

In [ ]:
import pandas as pd
import tensorflow as tf
csv1 = pd.read_csv("input_train.csv").values
csv2 = pd.read_csv("input_validate.csv").values
csv3 = pd.read_csv("input_test.csv").values

 Transform input-train to .TFRECORD format to facilitate efficient computing

In [ ]:
# Tfrecords format requires each record to be a tf example of {"inputs":tf.string, "targets":tf.string}

In [ ]:
with tf.io.TFRecordWriter("pegasus/data/testdata/input_train.tfrecord") as writer:
    for row in csv1:
        inputs, targets = row[:-1], row[-1]
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                   "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())

In [ ]:
with tf.io.TFRecordWriter("pegasus/data/testdata/input_validate.tfrecord") as writer:
    for row in csv2:
        inputs, targets = row[:-1], row[-1]
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                   "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())

In [ ]:
with tf.io.TFRecordWriter("pegasus/data/testdata/input_test.tfrecord") as writer:
    for row in csv3:
        inputs, targets = row[:-1], row[-1]
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                   "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())

Register the Fine-tuning dataset in /pegasus/params/public_params.py.

In [ ]:
# All the fine-tuning datasets must be registered before they are used for fine-tuning

In [ ]:
'''
@registry.register("new_params")
def new_params(param_overrides):
  return transformer_params(
      {
          "train_pattern": "tfrecord:/content/pegasus/pegasus/data/testdata/input_train.tfrecord",
          "dev_pattern": "tfrecord:/content/pegasus/pegasus/data/testdata/input_validate.tfrecord",
          "test_pattern": "tfrecord:/content/pegasus/pegasus/data/testdata/input_test.tfrecord",
          "max_input_len": 1024,
          "max_output_len": 256,
          "train_steps": 180000,
          "learning_rate": 0.0001,
          "batch_size": 8,
      }, param_overrides)
'''

In [ ]:
!pwd

/content/PEGASUS


In [ ]:
# Setting the TensorFlow loglevel to 3 to optimize run-time instance utilization

In [ ]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Fine-tuning the PEGASUS model with the newly registered data (WikiHow Knowledge base)

In [ ]:
# The newly registed fine-tuning dataset is passed as parameter to pegasus/bin/train.py --params=WikiHow
# The resultant fine-tuned model will be stored in the model_dir=ckpt/pegasus_ckpt/WikiHow folder

In [ ]:
!python3 pegasus/bin/train.py --params=WikiHow \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/WikiHow



W0801 13:22:52.098372 139802910123904 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0801 13:22:52.328975 139802910123904 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0801 13:22:52.329817 139802910123904 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7f25e580f950>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/wiki', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': No

Fine-tuned model evaluation

In [ ]:
# The fine-tuned model is evalauted on the registered validation and test data sets

The results of the model which includes the predictions(machine generated summaries) and the evaluation metrics are stored in the /ckpt/pegasus_ckpt/WikiHow

In [ ]:
# text_metrics-*.txt includes the various summarization metrics such as ROUGE, BLEU, and Length Statistics etc..
# inputs-.txt - contains the input data 
# targets-.txt - contains the actual summaries for a given set of text documents
# predictions-*.txt: contains the predicted summaries for a given set of text documents

# The prediction (summary generation) is written to the text file instead of the console as it takes more time. All the predicted summaries can be found in the 'predictions.txt' file.

In [ ]:
!python3 pegasus/bin/evaluate.py --params=WikiHow \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/WikiHow